# Cancer Prediction from Metabolic Biomarkers - Demo

**Model Version:** v0.2.0 (6 biomarkers)  
**Accuracy:** 99.21%  
**Based on:** Warburg Effect (altered cancer cell metabolism)

---

This notebook demonstrates how to use the metabolic cancer predictor model to predict cancer risk from routine blood test biomarkers.

## 1. Setup and Load Model

First, let's import the required libraries and load the v0.2.0 model.

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Libraries imported successfully")

In [ ]:
# Load the v0.2.0 model
with open('models/metabolic_cancer_predictor_v2.pkl', 'rb') as f:
    model_data = pickle.load(f)

model = model_data['model']
feature_names = model_data['features']
version = model_data['version']

# Get feature importance from the trained Random Forest model
feature_importance = model.feature_importances_

print(f"✅ Model loaded: version {version}")
print(f"📊 Biomarkers: {', '.join(feature_names)}")
print(f"🎯 Model type: {type(model).__name__}")

## 2. The 6 Biomarker Panel

Our model uses 6 routine blood test biomarkers, all available at standard clinical labs:

| # | Biomarker | Normal Range | Cancer Range | Importance |
|---|-----------|--------------|--------------|------------|
| 1 | **Glucose** | 4-6 mM | 5-7 mM | 31.97% |
| 2 | **Age** | Any | Older | 18.53% |
| 3 | **BMI** | 18.5-24.9 | Variable | 4.62% |
| 4 | **Lactate** | 0.5-2.2 mM | 2-5 mM | 15.27% |
| 5 | **LDH** | 140-280 U/L | 300-600 U/L | 24.73% |
| 6 | **CRP** | <10 mg/L | 10-100 mg/L | 4.88% |

**Total cost:** ~$150 per test

In [ ]:
# Visualize feature importance
plt.figure(figsize=(10, 6))
colors = ['#e74c3c' if imp > 0.15 else '#f39c12' if imp > 0.10 else '#3498db' for imp in feature_importance]
bars = plt.barh(feature_names, feature_importance * 100, color=colors)
plt.xlabel('Feature Importance (%)', fontsize=12)
plt.title('Biomarker Importance in Cancer Prediction (v0.2.0)', fontsize=14, fontweight='bold')
plt.xlim(0, 35)

# Add percentage labels
for i, (bar, imp) in enumerate(zip(bars, feature_importance)):
    plt.text(imp * 100 + 0.5, i, f'{imp*100:.2f}%', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n🔴 Red: Critical markers (>15%)")
print("🟡 Orange: Important markers (10-15%)")
print("🔵 Blue: Supporting markers (<10%)")

## 3. Example Predictions

Let's test the model with example patient data.

### Example 1: Healthy Individual

In [ ]:
# Healthy patient data (normal biomarkers)
healthy_patient = np.array([[
    5.2,    # Glucose (mM) - normal range
    45,     # Age (years) - middle-aged
    22.0,   # BMI (kg/m²) - normal weight
    1.5,    # Lactate (mM) - normal
    220,    # LDH (U/L) - normal
    5.0     # CRP (mg/L) - normal (low inflammation)
]])

# Make prediction
prediction = model.predict(healthy_patient)[0]
probability = model.predict_proba(healthy_patient)[0]

print("=" * 60)
print("HEALTHY PATIENT EXAMPLE")
print("=" * 60)
print(f"Glucose: {healthy_patient[0, 0]} mM (Normal: 4-6 mM)")
print(f"Age: {healthy_patient[0, 1]} years")
print(f"BMI: {healthy_patient[0, 2]} kg/m² (Normal: 18.5-24.9)")
print(f"Lactate: {healthy_patient[0, 3]} mM (Normal: 0.5-2.2 mM)")
print(f"LDH: {healthy_patient[0, 4]} U/L (Normal: 140-280 U/L)")
print(f"CRP: {healthy_patient[0, 5]} mg/L (Normal: <10 mg/L)")
print("\n" + "=" * 60)
print(f"PREDICTION: {'🟢 HEALTHY' if prediction == 0 else '🔴 CANCER'}")
print(f"Healthy probability: {probability[0]:.1%}")
print(f"Cancer probability: {probability[1]:.1%}")
print("=" * 60)

### Example 2: Cancer Patient (Warburg Effect)

In [ ]:
# Cancer patient data (elevated Warburg markers)
cancer_patient = np.array([[
    6.5,    # Glucose (mM) - elevated
    68,     # Age (years) - older
    21.0,   # BMI (kg/m²) - normal
    4.2,    # Lactate (mM) - ELEVATED (Warburg effect!)
    485,    # LDH (U/L) - ELEVATED (Warburg effect!)
    35.0    # CRP (mg/L) - elevated inflammation
]])

# Make prediction
prediction = model.predict(cancer_patient)[0]
probability = model.predict_proba(cancer_patient)[0]

print("=" * 60)
print("CANCER PATIENT EXAMPLE (Warburg Effect)")
print("=" * 60)
print(f"Glucose: {cancer_patient[0, 0]} mM (Normal: 4-6 mM) ⚠️ ELEVATED")
print(f"Age: {cancer_patient[0, 1]} years ⚠️ OLDER")
print(f"BMI: {cancer_patient[0, 2]} kg/m²")
print(f"Lactate: {cancer_patient[0, 3]} mM (Normal: 0.5-2.2 mM) 🔴 ELEVATED")
print(f"LDH: {cancer_patient[0, 4]} U/L (Normal: 140-280 U/L) 🔴 ELEVATED")
print(f"CRP: {cancer_patient[0, 5]} mg/L (Normal: <10 mg/L) ⚠️ ELEVATED")
print("\n" + "=" * 60)
print(f"PREDICTION: {'🟢 HEALTHY' if prediction == 0 else '🔴 CANCER'}")
print(f"Healthy probability: {probability[0]:.1%}")
print(f"Cancer probability: {probability[1]:.1%}")
print("=" * 60)
print("\n📊 Warburg Effect Markers:")
print(f"   Lactate: {cancer_patient[0, 3]} mM (2.8x normal upper limit)")
print(f"   LDH: {cancer_patient[0, 4]} U/L (1.7x normal upper limit)")
print("   → Indicates aerobic glycolysis (cancer metabolism)")

### Example 3: Borderline Case (Diabetes with Inflammation)

In [ ]:
# Borderline case - diabetes with inflammation (confounding factors)
borderline_patient = np.array([[
    7.0,    # Glucose (mM) - elevated (diabetes)
    55,     # Age (years)
    28.5,   # BMI (kg/m²) - overweight
    2.5,    # Lactate (mM) - slightly elevated
    310,    # LDH (U/L) - slightly elevated
    18.0    # CRP (mg/L) - moderately elevated
]])

# Make prediction
prediction = model.predict(borderline_patient)[0]
probability = model.predict_proba(borderline_patient)[0]

print("=" * 60)
print("BORDERLINE CASE (Diabetes + Inflammation)")
print("=" * 60)
print(f"Glucose: {borderline_patient[0, 0]} mM (Normal: 4-6 mM) ⚠️ ELEVATED (diabetes)")
print(f"Age: {borderline_patient[0, 1]} years")
print(f"BMI: {borderline_patient[0, 2]} kg/m² ⚠️ OVERWEIGHT")
print(f"Lactate: {borderline_patient[0, 3]} mM (Normal: 0.5-2.2 mM) ⚠️ SLIGHTLY HIGH")
print(f"LDH: {borderline_patient[0, 4]} U/L (Normal: 140-280 U/L) ⚠️ SLIGHTLY HIGH")
print(f"CRP: {borderline_patient[0, 5]} mg/L (Normal: <10 mg/L) ⚠️ ELEVATED")
print("\n" + "=" * 60)
print(f"PREDICTION: {'🟢 HEALTHY' if prediction == 0 else '🔴 CANCER'}")
print(f"Healthy probability: {probability[0]:.1%}")
print(f"Cancer probability: {probability[1]:.1%}")
print("=" * 60)
print("\n💡 Clinical Note:")
print("   Elevated markers may be due to diabetes and obesity.")
print("   Model considers ALL biomarkers together to distinguish")
print("   cancer-specific metabolic patterns from confounding conditions.")

## 4. Model Performance on Test Data

Let's evaluate the model on the full test dataset.

In [ ]:
# Load test data
test_data = np.load('data/test_data_v2.npz')
X_test = test_data['X']
y_test = test_data['y']

print(f"✅ Test data loaded: {len(X_test):,} samples")
print(f"   - Healthy: {np.sum(y_test == 0):,} samples")
print(f"   - Cancer: {np.sum(y_test == 1):,} samples")

In [ ]:
# Make predictions on test set
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_proba)

print("=" * 60)
print("MODEL PERFORMANCE ON TEST DATA (v0.2.0)")
print("=" * 60)
print(f"Test Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision:      {precision:.4f} ({precision*100:.2f}%)")
print(f"Recall/Sens.:   {recall:.4f} ({recall*100:.2f}%)")
print(f"F1 Score:       {f1:.4f}")
print(f"AUC-ROC:        {auc_score:.4f}")
print("=" * 60)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[0],
            xticklabels=['Healthy', 'Cancer'], yticklabels=['Healthy', 'Cancer'])
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('Actual', fontsize=12)
axes[0].set_title('Confusion Matrix', fontsize=14, fontweight='bold')

# Add text annotations
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)
axes[0].text(0.5, -0.15, f'True Negatives: {tn:,} | False Positives: {fp:,}\nFalse Negatives: {fn:,} | True Positives: {tp:,}',
             transform=axes[0].transAxes, ha='center', fontsize=10)

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
axes[1].plot(fpr, tpr, color='#e74c3c', linewidth=2, label=f'AUC = {auc_score:.4f}')
axes[1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
axes[1].set_xlabel('False Positive Rate', fontsize=12)
axes[1].set_ylabel('True Positive Rate (Sensitivity)', fontsize=12)
axes[1].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower right', fontsize=10)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Key Performance Indicators:")
print(f"   Sensitivity (Recall): {recall*100:.2f}% - catches {recall*100:.2f}% of cancers")
print(f"   Specificity: {specificity*100:.2f}% - correctly identifies {specificity*100:.2f}% of healthy")
print(f"   False Negative Rate: {(1-recall)*100:.4f}% - misses only {fn} out of {tp+fn:,} cancers")
print(f"   False Positive Rate: {(1-specificity)*100:.2f}% - {fp} out of {tn+fp:,} healthy flagged")

## 5. Interactive Prediction Tool

Try your own biomarker values!

In [ ]:
def predict_cancer_risk(glucose, age, bmi, lactate, ldh, crp):
    """
    Predict cancer risk from biomarker values.
    
    Parameters:
    -----------
    glucose : float
        Glucose in mM (normal: 4-6)
    age : int
        Age in years
    bmi : float
        Body Mass Index in kg/m² (normal: 18.5-24.9)
    lactate : float
        Lactate in mM (normal: 0.5-2.2)
    ldh : float
        LDH in U/L (normal: 140-280)
    crp : float
        CRP in mg/L (normal: <10)
    """
    # Create input array
    patient_data = np.array([[glucose, age, bmi, lactate, ldh, crp]])
    
    # Make prediction
    prediction = model.predict(patient_data)[0]
    probability = model.predict_proba(patient_data)[0]
    
    # Display results
    print("=" * 60)
    print("CANCER RISK PREDICTION")
    print("=" * 60)
    print(f"Input Biomarkers:")
    print(f"  Glucose: {glucose} mM {'✅' if 4 <= glucose <= 6 else '⚠️'}")
    print(f"  Age: {age} years")
    print(f"  BMI: {bmi} kg/m² {'✅' if 18.5 <= bmi <= 24.9 else '⚠️'}")
    print(f"  Lactate: {lactate} mM {'✅' if 0.5 <= lactate <= 2.2 else '⚠️'}")
    print(f"  LDH: {ldh} U/L {'✅' if 140 <= ldh <= 280 else '⚠️'}")
    print(f"  CRP: {crp} mg/L {'✅' if crp < 10 else '⚠️'}")
    print("\n" + "=" * 60)
    print(f"PREDICTION: {'🟢 HEALTHY' if prediction == 0 else '🔴 CANCER'}")
    print(f"\nProbabilities:")
    print(f"  Healthy: {probability[0]:.1%}")
    print(f"  Cancer:  {probability[1]:.1%}")
    print("=" * 60)
    
    # Risk interpretation
    if probability[1] < 0.2:
        risk_level = "LOW"
        color = "🟢"
    elif probability[1] < 0.5:
        risk_level = "MODERATE"
        color = "🟡"
    elif probability[1] < 0.8:
        risk_level = "HIGH"
        color = "🟠"
    else:
        risk_level = "VERY HIGH"
        color = "🔴"
    
    print(f"\nRisk Level: {color} {risk_level}")
    
    return prediction, probability

# Example usage
print("Example: Testing a patient with slightly elevated markers\n")
predict_cancer_risk(
    glucose=6.0,
    age=60,
    bmi=23.5,
    lactate=2.8,
    ldh=320,
    crp=15.0
)

### Try Your Own Values

Uncomment and modify the values below to test different scenarios:

In [ ]:
# Uncomment and modify these values to test:

# predict_cancer_risk(
#     glucose=5.5,    # Your glucose value (mM)
#     age=50,         # Your age (years)
#     bmi=22.0,       # Your BMI (kg/m²)
#     lactate=1.8,    # Your lactate value (mM)
#     ldh=200,        # Your LDH value (U/L)
#     crp=5.0         # Your CRP value (mg/L)
# )

## 6. Understanding the Warburg Effect

The model is based on the **Warburg effect** - a metabolic shift in cancer cells:

### Normal Cell Metabolism
```
Glucose → [Glycolysis] → Pyruvate → [Mitochondria] → CO₂ + H₂O + Energy (36 ATP)
```

### Cancer Cell Metabolism (Warburg Effect)
```
Glucose → [Glycolysis] → Pyruvate → [LDH] → Lactate + Energy (2 ATP)
                                              ↓
                                    (Even with oxygen present!)
```

### What This Means for Biomarkers:

1. **Increased Glucose** consumption (cancer cells need more fuel)
2. **Elevated Lactate** production (end product of glycolysis)
3. **High LDH** enzyme levels (catalyzes pyruvate → lactate conversion)
4. **Elevated CRP** (tumor-associated inflammation)

These metabolic changes create a **detectable signature** in routine blood tests!

## 7. Important Limitations

⚠️ **This model is for RESEARCH ONLY:**

- ❌ NOT FDA approved
- ❌ NOT for clinical diagnosis
- ❌ Trained on synthetic data (not real patients)
- ❌ Requires real-world validation
- ❌ Cannot replace standard cancer screening

### Expected Performance on Real Data:

- **Synthetic test data:** 99.21% accuracy ✅
- **Expected real-world:** 85-95% accuracy ⏳
- **Validation needed:** MIMIC-IV clinical database (pending access)

### Use Cases:

✅ Research and education  
✅ Proof-of-concept for metabolic cancer screening  
✅ Feature importance analysis  
❌ Clinical decision making  
❌ Patient diagnosis  

## 8. Model Comparison: v0.1.0 vs v0.2.0

| Metric | v0.1.0 (7 biomarkers) | v0.2.0 (6 biomarkers) | Change |
|--------|----------------------|----------------------|--------|
| **Test Accuracy** | 99.20% | **99.21%** | +0.01% ✅ |
| **Sensitivity** | 99.96% | **99.98%** | +0.02% ✅ |
| **False Negatives** | 2 | **1** | -50% ✅ |
| **Cost per test** | ~$175 | **~$150** | -$25 ✅ |
| **Biomarkers** | 7 | **6** | Simpler ✅ |

**Removed:** Specific Gravity (contributed only 1.26% importance)

**Result:** Better performance, lower cost, simpler model!

## 9. Next Steps

### For Researchers:
1. Explore the training data generation script: `src/generate_synthetic_data.py`
2. Review feature importance analysis: `FEATURE_IMPORTANCE_SUMMARY.md`
3. Check external validation: `UCI_TEST_RESULTS_EXPLAINED.md`

### For Developers:
1. Load and use the model in your applications
2. Retrain with your own data: `src/train_model.py`
3. Contribute improvements via GitHub

### For Validation:
1. Test on real patient data (with proper IRB approval)
2. Compare with standard cancer screening methods
3. Publish validation results

---

**Repository:** https://github.com/rodtjarn/cancer_predictor  
**License:** MIT  
**Model Version:** 0.2.0  
**Last Updated:** 2025-12-31